<a href="https://colab.research.google.com/github/i2mmmmm/kaggle_flood/blob/main/3.%20feature_engineering_lgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_USERNAME']='mmmmmlee'
os.environ['KAGGLE_KEY']='b4527f98e27813c6dae3fb199096dcb5'

In [2]:
!kaggle competitions download -c Playground-series-s4e5

 50% 14.0M/28.0M [00:00<00:00, 146MB/s]
100% 28.0M/28.0M [00:00<00:00, 167MB/s]


In [3]:
!unzip 'Playground-series-s4e5.zip'

Archive:  Playground-series-s4e5.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.ensemble import VotingRegressor, BaggingRegressor, StackingRegressor
from sklearn.linear_model import BayesianRidge, SGDRegressor, Ridge
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import make_pipeline
import xgboost as xgb
import lightgbm as lgb


In [53]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [54]:
# train 표준화, 스케일링
scaler = StandardScaler()
X = train.drop(columns=['FloodProbability', 'id'])
y = train['FloodProbability']
X_scaled = scaler.fit_transform(X)

In [55]:
# 선형회귀_baseline

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=35)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('R2 Score:', r2_score(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))

R2 Score: 0.8449835489068076
MSE: 0.00040395088979260915


In [56]:
# Feature Engineering
train['Climate_Risk'] = train['MonsoonIntensity'] * train['ClimateChange']
train['Infrastructure_Risk'] = train['DamsQuality'] * train['DrainageSystems']
train['wet_Risk'] = train['WetlandLoss'] + train['Encroachments']

In [57]:
# 다시 스케일링
X = train.drop(columns=['FloodProbability', 'id'])
y = train['FloodProbability']
X_scaled = scaler.fit_transform(X)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=35)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('R2 Score:', r2_score(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))

R2 Score: 0.8449901321915165
MSE: 0.00040393373468618546


In [61]:
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'SGD Regressor': SGDRegressor(),
    'Bayesian Ridge Regression': BayesianRidge(),
    'Bagging Regressor': BaggingRegressor(estimator=LinearRegression()),
    'XGBoost Regressor': xgb.XGBRegressor(),
    'LightGBM Regressor': lgb.LGBMRegressor(),
}

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=35)

kf = KFold(n_splits=5, shuffle=True, random_state=35)

for name, model in models.items():
    mse_values = []
    r2_values = []
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        r2_values.append(r2_score(y_val, y_pred))
    avg_r2 = np.mean(r2_values)
    print(f'{name}: R2 = {avg_r2:.4f}')

Linear Regression: R2 = 0.8450
Ridge Regression: R2 = 0.8450
SGD Regressor: R2 = 0.8068
Bayesian Ridge Regression: R2 = 0.8450


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Bagging Regressor: R2 = 0.8449
XGBoost Regressor: R2 = 0.8137
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.210759 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 527
[LightGBM] [Info] Number of data points in the train set: 894365, number of used features: 23
[LightGBM] [Info] Start training from score 0.504465
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.222365 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 525
[LightGBM] [Info] Number of data points in the train set: 894365, number of used features: 23
[LightGBM] [Info] Start training from score 0.504491
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.204799 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 527
[LightGBM] [Info] Number of data points in the train set: 8

Linear Regression: R2 = 0.8450  
Ridge Regression: R2 = 0.8450  
SGD Regressor: R2 = 0.8068  
Bayesian Ridge Regression: R2 = 0.8450  
Bagging Regressor: R2 = 0.8449  
XGBoost Regressor: R2 = 0.8137  
LightGBM Regressor: R2 = 0.7876  

In [65]:
# 다시 feature engineering
train = pd.read_csv('train.csv')

def simplified_getFeats(df):
    num_cols = ['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement', 'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality', 'Siltation', 'AgriculturalPractices', 'Encroachments', 'IneffectiveDisasterPreparedness', 'DrainageSystems', 'CoastalVulnerability', 'Landslides', 'Watersheds', 'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss', 'InadequatePlanning', 'PoliticalFactors']

    scaler = StandardScaler().fit(df[num_cols])
    df[num_cols] = scaler.transform(df[num_cols])  # Scale early

    # Basic Stats
    df['mean'] = df[num_cols].mean(axis=1)
    df['std'] = df[num_cols].std(axis=1)
    df['max'] = df[num_cols].max(axis=1)
    df['min'] = df[num_cols].min(axis=1)

    # Interaction Features (Simplified)
    df['Climate_Risk'] = df['MonsoonIntensity'] * df['ClimateChange']
    df['Infrastructure_Risk'] = df['DamsQuality'] * df['DrainageSystems']
    df['wet_Risk'] = df['WetlandLoss'] + df['Encroachments']

    return df

train_simp = simplified_getFeats(train)

In [66]:
train_simp.head()

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,InadequatePlanning,PoliticalFactors,FloodProbability,mean,std,max,min,Climate_Risk,Infrastructure_Risk,wet_Risk
0,0,0.038198,1.467769,0.021561,1.490363,0.507578,-0.453941,-0.458881,-0.933107,-0.939124,...,0.989569,-0.927598,0.445,-0.115050,0.844049,1.490363,-1.417247,-0.017340,-0.011760,-0.431878
1,1,0.524488,0.990186,-0.461022,-0.459251,1.467552,1.489938,-0.938943,0.034951,-0.455692,...,-0.451962,-0.927598,0.450,-0.114543,1.105743,1.951149,-2.372809,0.781455,-0.930233,-0.438667
2,2,0.524488,0.035021,0.504143,1.002959,-0.932383,1.003968,-1.899068,0.034951,-0.455692,...,-0.932472,-0.927598,0.530,0.007501,0.933698,1.481302,-1.899068,0.526569,-1.881450,-1.402065
3,3,-0.934382,-0.442562,0.504143,0.028152,-0.452396,1.489938,-0.458881,1.003010,0.511171,...,0.989569,0.029180,0.535,0.126028,0.792178,1.489938,-1.422018,-1.392172,0.652537,1.488131
4,4,0.038198,-0.920145,-1.426186,0.515556,-0.452396,-0.453941,-0.938943,-0.933107,-0.939124,...,-0.932472,0.029180,0.415,-0.644937,0.725208,0.515556,-1.893533,-0.017340,1.335194,-2.362070


In [68]:
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'SGD Regressor': SGDRegressor(),
    'Bayesian Ridge Regression': BayesianRidge(),
    'Bagging Regressor': BaggingRegressor(estimator=LinearRegression()),
    'XGBoost Regressor': xgb.XGBRegressor(),
    'LightGBM Regressor': lgb.LGBMRegressor(),
}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=35)

kf = KFold(n_splits=5, shuffle=True, random_state=35)

for name, model in models.items():
    mse_values = []
    r2_values = []
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        r2_values.append(r2_score(y_val, y_pred))
    avg_r2 = np.mean(r2_values)
    print(f'{name}: R2 = {avg_r2:.4f}')

Linear Regression: R2 = 0.8451
Ridge Regression: R2 = 0.8451
SGD Regressor: R2 = 0.8444
Bayesian Ridge Regression: R2 = 0.8451
Bagging Regressor: R2 = 0.8451
XGBoost Regressor: R2 = 0.8663
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.235419 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1854
[LightGBM] [Info] Number of data points in the train set: 894365, number of used features: 27
[LightGBM] [Info] Start training from score 0.504465
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.409141 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1862
[LightGBM] [Info] Number of data points in the train set: 894365, number of used features: 27
[LightGBM] [Info] Start training from score 0.504491
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.235717 seconds.
You can set `force_co

Linear Regression: R2 = 0.8451 (이전 : 0.8450)  
Ridge Regression: R2 = 0.8451 (이전 : 0.8450)   
SGD Regressor: R2 = 0.8444 (이전 : 0.8068)  
Bayesian Ridge Regression: R2 = 0.8451 (이전 : 0.8450)  
Bagging Regressor: R2 = 0.8451 (이전 : 0.8449)  
XGBoost Regressor: R2 = 0.8663 (이전 : 0.8137)  
LightGBM Regressor: R2 = 0.8663 (이전 : 0.7876)  

In [71]:
# test에도 적용
test_simp = simplified_getFeats(test)

In [73]:
# test flood 예측 진행

X_train = train_simp.drop(columns=['FloodProbability', 'id'])
y_train = train_simp['FloodProbability']
X_test = test_simp.drop(columns=['id'])

In [74]:
# LightGBM 모델 학습

model_lgb = lgb.LGBMRegressor()
model_lgb.fit(X_train, y_train)
y_pred_lgb = model_lgb.predict(X_test)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.048439 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1857
[LightGBM] [Info] Number of data points in the train set: 1117957, number of used features: 27
[LightGBM] [Info] Start training from score 0.504480


In [78]:
# 출력
test_simp['FloodProbability'] = y_pred_lgb
test_simp[['id', 'FloodProbability']].to_csv('submission2.csv', index=False)

# 특성 엔지니어링 이후 제출 - 0.866